In [ ]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 

In [ ]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest


def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10, 10))
    row = scaler.fit_transform(row)
    return row


def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = 0
        if data[i][3] > data[i][0]:
            sugg = 1

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
         data = yf.download(symbol, period="730d",
                            interval="1h", progress=False)
         if data.empty:
              pass
         else:
               if data.shape[0] > 12:
                   unattached_dfs.append(
                       process(data))

    data = np.array(unattached_dfs[0])
    for i in unattached_dfs[1:]:
        data = np.append(data, np.array(i), axis=0)
        
    symbol = []

    data = pd.DataFrame(data, columns=clmns)
    data = data.astype(float)
    data = data.astype({'suggestion': 'int'})

    # xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,10000)}.csv")
    # return xTrain, xTest, yTrain, yTest,data


# symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
# symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]

# symbols = read_syms_from_txt()
symbols = get_crypto_syms()

pieces = 15
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays[7:]:
    get_data(symbols)

# xTrain, xTest, yTrain, yTest,data= get_data(symbols)
# data 

In [ ]:
data = []
data = pd.concat([pd.read_parquet("/content/drive/MyDrive/lol1.parquet"),pd.read_parquet("/content/drive/MyDrive/lol1.parquet")])
data.drop("Unnamed: 0",axis=1,inplace=True)
xTrain, xTest, yTrain, yTest= spliting(data)
data = []

(5741836, 66) (5741836,)
(637982, 66) (637982,)


In [ ]:
model = Sequential()



model.add(Dense(3000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2000, activation='relu'))
model.add(Dense(1750, activation='relu'))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1250, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 3000)              201000    
                                                                 
 dense_8 (Dense)             (None, 2000)              6002000   
                                                                 
 dense_9 (Dense)             (None, 1750)              3501750   
                                                                 
 dense_10 (Dense)            (None, 1500)              2626500   
                                                                 
 dense_11 (Dense)            (None, 1250)              1876250   
                                                                 
 dense_12 (Dense)            (None, 1000)              1251000   
                                                                 
 dense_13 (Dense)            (None, 1000)             

In [ ]:
model.fit(xTrain,yTrain,epochs=35,batch_size=12345,validation_data=(xTest,yTest))

Epoch 1/35
575/575 [==============================] - 286s 489ms/step - loss: 0.7119 - accuracy: 0.5301 - val_loss: 0.6863 - val_accuracy: 0.5445
Epoch 2/35
575/575 [==============================] - 282s 491ms/step - loss: 0.6806 - accuracy: 0.5571 - val_loss: 0.6748 - val_accuracy: 0.5665
Epoch 3/35
575/575 [==============================] - 282s 490ms/step - loss: 0.6673 - accuracy: 0.5779 - val_loss: 0.6633 - val_accuracy: 0.5826
Epoch 4/35
575/575 [==============================] - 282s 490ms/step - loss: 0.6529 - accuracy: 0.5950 - val_loss: 0.6518 - val_accuracy: 0.5949
Epoch 5/35
575/575 [==============================] - 283s 491ms/step - loss: 0.6353 - accuracy: 0.6140 - val_loss: 0.6341 - val_accuracy: 0.6141
Epoch 6/35
575/575 [==============================] - 283s 493ms/step - loss: 0.6124 - accuracy: 0.6379 - val_loss: 0.6154 - val_accuracy: 0.6344
Epoch 7/35
575/575 [==============================] - 283s 493ms/step - loss: 0.5813 - accuracy: 0.6689 - val_loss: 0.5910 -

In [ ]:
model.save(f"{model.evaluate(xTest,yTest)}.h5")

In [ ]:
model.save("fffirst_model.h5")
model.fit(xTrain,yTrain,epochs=10,batch_size=256,validation_data=(xTest,yTest))
model.save("sssecond_model.h5")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
